In [ ]:
import pyarrow.parquet as pq
import pandas as pd

# clientes = pq.read_table('0clientes.parquet').to_pandas()
saldos = pq.read_table('0saldos.parquet').to_pandas()
saldos = saldos.loc[saldos['TipoDocum'] == 'C ']
# clientes = clientes.loc[clientes['TIPODOCUM'] == 'C']
# saldos
conteo_contratos_por_cliente = saldos.groupby('NroDocum').agg(
    # Numero_de_Contratos_Activos=('Contrato', 'count'),  # Conteo de contratos por cliente
    Contrato=('Contrato', list),
    Saldo2022Aug=('SALDO_202210', list),
    # NumeroProductos=('PlanProducto', 'count'),  # Conteo de productos por contrato
    Productos=('PlanProducto', list)
)


# print(conteo_contratos_por_cliente.head())
# missings = set(saldos.NroDocum.drop_duplicates().values) - set(clientes.NroDocum.values)
# print(missings)



In [81]:
from datetime import datetime as dt
import numpy

transferencias = pq.read_table('0transferencias.parquet').to_pandas()
# transferencias.nunique()
transferencias.drop(
    columns=['Usuario', 'CodDestino', 'CodTipoGiro', 'CodBancoDestinoACH', 'CodBancoSkandia', 'TipoCtaACH', 'TipoOper',
             'EstadoReverso'], inplace=True)
transferencias.sort_values(by='FechaEfectiva', ascending=True, inplace=True)
transferencias.FechaEfectiva = pd.to_datetime(transferencias.FechaEfectiva, format='%Y-%m-%d')

# dt(2019, 1, 1)
# print(transferencias.FechaEfectiva.drop_duplicates().values)

# Agrupar las transacciones por mes y sumar los valores
transferencias = transferencias.groupby(
    [transferencias['Contrato'], transferencias['FechaEfectiva'].dt.to_period('M')], as_index=False
).agg(
    FechaEfectiva=('FechaEfectiva', 'first'),
    ValorTransaccion=('ValorNeto', 'sum'),
    Productos=('PlanProducto', list)
)
transferencias['FechaEfectiva'] = transferencias['FechaEfectiva'].dt.strftime('%Y-%m-01')
transferencias['Productos'] = transferencias['Productos'].apply(lambda x: list(set(x)))


,Contrato,FechaEfectiva,ValorTransaccion,Productos
180926,8774634,2007-07-01,-2.504880e+06,[7256]
180927,8774634,2007-09-01,-2.813080e+05,[7256]
180928,8774634,2008-04-01,-6.010790e+05,[7256]
180929,8774634,2009-01-01,-8.999500e+04,[7256]
180930,8774634,2010-09-01,-6.781670e+05,[7256]
...,...,...,...,...
180996,8774634,2022-06-01,3.748732e+08,[7256]
180997,8774634,2022-07-01,2.882305e+08,[7256]
180998,8774634,2022-08-01,7.750249e+08,[7256]
180999,8774634,2022-09-01,7.055359e+08,[7256]
